In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import yaml
from collections import defaultdict
import pprint

### Import column dictionary from yaml

The yaml file has information regarding the data in the following form:

  ```
  dictionary:
      school.name:
        source: INSTNM
        type: autocomplete
        description: Institution name
        index: fulltext 
  ```
The type indicates the data type and should be indicated when reading in the college scorecard csv.  
Null values are written as ```NULL``` or ```PrivacySupressed```.   
The dictionary is nested. For the example above, the ```school.name``` key returns four more keys ```source, type, description, index```. Each of those keys returns a value. The colum name is under the nested key ```source```. It needs to be flipped so that the value of ```source``` is the main key.

In [86]:
yaml_file = open("data.yaml", 'r')
yaml_content = yaml.load(yaml_file)

## Need to flip the dictionary so that the the keys match the column names
flippeddict = defaultdict(dict)
try:
    for key, val in yaml_content["dictionary"].items():
        newkey = val['source']
        for subkey, subval in val.items():
            flippeddict[newkey][subkey] = subval
except KeyError:
    pass

# pprint.pprint(dict(flippeddict))

<ipython-input-86-4fce6df8588d>:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_content = yaml.load(yaml_file)


*Work in progress, defining columns of interest*

In [164]:
cols_of_interest = ['INSTNM',  
                    'COSTT4_A', 'COSTT4_P', 'DEBT_MDN','MN_EARN_WNE_P6','MN_EARN_WNE_P8','MN_EARN_WNE_P10','TUITFTE', 'AVGFACSAL', 'ADM_RATE_ALL', 'SATVR25', 'SATVR75', 'SATMT25', 'SATMT75', 'ACTCM25', 
                    'ACTCM75', 'UGDS', 'UG25ABV', 'PCTFLOAN', 'CDR3' ]
cols_numeric = ['COSTT4_A', 'COSTT4_P', 'DEBT_MDN','MN_EARN_WNE_P6','MN_EARN_WNE_P8','MN_EARN_WNE_P10','TUITFTE', 'AVGFACSAL', 'ADM_RATE_ALL', 'SATVR25', 'SATVR75', 'SATMT25', 'SATMT75', 'ACTCM25', 
                    'ACTCM75', 'UGDS', 'UG25ABV', 'PCTFLOAN', 'CDR3' ]

# Import data

First a function is defined to read and merge csv files. Next the fucntion is run from the year 1996-2020.   
<div class="alert-danger">
To do: use yaml file to pipe in dtypes for each column
</div>

In [148]:
# Define a function read and merge all years
def read_cs_data(year,datadir):
    """read a CollegeScorecard dataframe"""
    nextyr = str(int(year) + 1)[-2:]
    filename = datadir + '/MERGED{}_{}_PP.csv'.format(year,nextyr)
    
    # Specify dtype because guessing dtypes is very memory intensive
    # Specify that "PrivacySuppressed" are NaN values
    # Eventually can use usecols to specify which columns of interest to import
    col = pd.read_csv(filename, dtype='unicode',na_values='PrivacySuppressed') 
    col['Year'] = pd.Period(str(int(year) + 1),freq='Y')
    return col

In [149]:
datadir = "/Users/jhuang/DS4A/CollegeScorecard_Raw_Data_08032021"
df = pd.concat((read_cs_data(str(y),datadir) for y in range(1996,2019)))
df = df.set_index(['UNITID','Year'])
df_subset = df[cols_of_interest]

In [92]:
df.head()

,UNITID,Year,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,...,FEDSCHCD,BOOKSUPPLY,ROOMBOARD_ON,OTHEREXPENSE_ON,ROOMBOARD_OFF,OTHEREXPENSE_OFF,OTHEREXPENSE_FAM,ENDOWBEGIN,ENDOWEND,DOLPROVIDER
0,100636,1997,01230800,012308,Community College of the Air Force,Montgomery,AL,36114-3011,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100654,1997,00100200,001002,Alabama A & M University,Normal,AL,35762,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100663,1997,00105200,001052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100672,1997,00574900,005749,ALABAMA AVIATION AND TECHNICAL COLLEGE,OZARK,AL,36360,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100690,1997,02503400,025034,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Save merged csv to disk

Next time, this merged csv can be read directly to skip the time-consuming step of reading each year separately.

In [150]:
df.to_csv('ALLMERGED.csv') # Export to csv

In [168]:
## Use this to read in merged csv directly
df = pd.read_csv('ALLMERGED.csv', dtype='unicode',na_values='PrivacySuppressed') 

# Data cleaning

To begin, columns with more than 60% ```NaN``` values are omitted. This can percentage can be changed in the code. Three variables related to earnings post-graduation are added back in, ```MN_EARN_WNE_P6,MN_EARN_WNE_P8,MN_EARN_WNE_P10```. There are likely more that can be added back easily.

### Dropping columns with too many ```NaN```

In [174]:
# Function to get a table of missing values
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns

In [175]:
mis_val = missing_values_table(df)
mis_val.head()

Your selected dataframe has 2393 columns.
There are 2381 columns that have missing values.


,Missing Values,% of Total Values
ACCREDAGENCY,163332,100.0
DTRANS_4_POOLED,163332,100.0
D100_4_POOLED,163332,100.0
C100_L4_POOLED,163332,100.0
C100_4_POOLED,163332,100.0


In [176]:
# Drop columms with more than 60% NaN
df_dropna = df.dropna(thresh=df.shape[0]*0.6,how='all',axis=1)

# Add back in certain columns
cols_to_add = ['MN_EARN_WNE_P6','MN_EARN_WNE_P8','MN_EARN_WNE_P10']
for col in cols_to_add:
    df_dropna[col] = df[col]

<ipython-input-176-056a24831277>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropna[col] = df[col]


In [191]:
print("Shape before:", df.shape, " Shape after:", df_dropna.shape)

myfile = open('dropna_columns.txt', 'w')
for col in df_dropna.columns:
    try:
#         print(col,":", flippeddict[col]["description"])
        var1=col+":"+" "+flippeddict[col]["description"]
        myfile.write("%s\n" % var1)
    except KeyError:
        pass
myfile.close()

Shape before: (163332, 2393)  Shape after: (163332, 350)


### Use dictionary from yaml to change column dtypes

Must do this after defining columns of interest. It is too time consuming to do it for the whole dataset.

In [182]:
# Function to set dtypes on columns of interest in a dataframe using the yaml flipped dictionary from
def setdtypes(cols_of_interest, df, flippeddict):
    for col in cols_of_interest:
        print("Doing ",cols_of_interest)
        try:
            if flippeddict[col]["type"] == "integer":
                df[col] = df[col].astype("int")
                print("dtype changed to int.")
            elif flippeddict[col]["type"] == "float":
                df[col] = df[col].astype("float")
                print("dtype changed to float.")
            else:
                pass
        except KeyError:
            print("KeyError, skipping")
            pass
        except ValueError:
            print("ValueError, skipping")
            pass
        
#     print(flippeddict[col])

In [ ]:
# df["ACTCM25"] = df['PCTFLOAN'].astype('float')
# df["ADM_RATE_ALL"].dtypes

# Determining feature importance

Going to try to use some models to determine which features are important. Work in progress.

In [76]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer

model=c(random_state=1)
 
features=df
 
model.fit(features,df_dropna['MN_EARN_WNE_P6'])
 
feature_importance=pd.DataFrame({'feature':list(features.columns),'feature_importance':[abs(i) for i in model.coef_[0]]})
feature_importance.sort_values('feature_importance',ascending=False)
 

ValueError: could not convert string to float: '001057A1'

### Plotting stuff

In [ ]:
df_subset = df[cols_of_interest]
for col in cols_numeric:
    df_subset[col] = pd.to_numeric(df_subset[col], errors='coerce')
df_subset.info()

In [ ]:
sns.histplot(df_subset, y="TUITFTE", x="DEBT_MDN", bins=50, pthresh=.1, cmap="mako")
plt.title("Cost of tuition vs Median earnings after 6 years")